In [1]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

/Users/shaikhygalizhassulan/Library/Python/3.9/lib/python/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/shaikhygalizhassulan/terraform-key.json"


In [5]:
# Initialize a BigQuery client
project_id = "sustainable-data-platform"
client = bigquery.Client(project=project_id)

# Define the table ID
table_id = "sustainable-data-platform.sdp.sample_sustainability_data_with_schema"

# Attempt to delete the table if it exists
try:
    client.delete_table(table_id, not_found_ok=True)  # Make an API request.
    print(f"Deleted table '{table_id}'.")
except NotFound:
    print(f"Table '{table_id}' not found.")

Deleted table 'sustainable-data-platform.sdp.sample_sustainability_data_with_schema'.


In [6]:




# Define the job configuration for loading data
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,  # Skip the header row
    autodetect=True,  # Automatically detect the schema
)

# Define the URI of the source file in GCS
uri = "gs://sdp-dev/sample_sustainability_*.csv"

# Start the load job
load_job = client.load_table_from_uri(
    uri,
    table_id,
    job_config=job_config
)

# Wait for the load job to complete
load_job.result()

print(f"Table '{table_id}' is updated with data from '{uri}'.")

Table 'sustainable-data-platform.sdp.sample_sustainability_data_with_schema' is updated with data from 'gs://sdp-dev/sample_sustainability_*.csv'.
